### 1. Merge the crawled tables into one

In [3]:
import pandas as pd
excel_files = ['search_0.xlsx', 'search_189.xlsx', 'search_378.xlsx','search_567.xlsx','search_756.xlsx','search_945.xlsx']

merged_data = pd.DataFrame()


for file in excel_files:
    df = pd.read_excel(file)
    merged_data = pd.concat([merged_data, df], ignore_index=True)
merged_data.to_excel('merged_artifacts.xlsx', index=False)


In [4]:
df = pd.read_excel('merged_artifacts.xlsx')
df.shape

(13722, 19)

In [4]:
df.columns

Index(['Name', 'Provenance', 'Collection', 'Previous custodian or owner',
       'Origin', 'Credit Line', 'Type', 'Restrictions and Rights', 'Period',
       'Geography', 'Material', 'Dimension', 'EDAN ID', '188', '377', '566',
       '755', '944', '1133'],
      dtype='object')

### Check the columns "Credit Line" and "Collection"

I use codes to check if all values in the "Credit Line" column are 'Gift of Charles Lang Freer'. Objetcs with inconsistency might not be collections donated by Freer.

In [5]:

# Check the column "Credit Line"
credit_line_check = all(df['Credit Line'] == 'Gift of Charles Lang Freer')

# Check the column "Collection"
collection_check = all(df['Collection'] == 'Freer Gallery of Art Collection')

# Print the results
if credit_line_check and collection_check:
    print("Both 'Credit Line' and 'Collection' values are consistent.")
else:
    print("There are inconsistencies in 'Credit Line' or 'Collection' values.")

There are inconsistencies in 'Credit Line' or 'Collection' values.


In [13]:
# Filter inconsistent rows
inconsistent_rows = df[(df['Credit Line'] != 'Gift of Charles Lang Freer')]

print(inconsistent_rows['Credit Line'])

2265                                         NaN
2375                                         NaN
2391                                         NaN
2811                                         NaN
4679                                         NaN
                          ...                   
13717    Purchase — Charles Lang Freer Endowment
13718    Purchase — Charles Lang Freer Endowment
13719    Purchase — Charles Lang Freer Endowment
13720    Purchase — Charles Lang Freer Endowment
13721    Purchase — Charles Lang Freer Endowment
Name: Credit Line, Length: 3856, dtype: object


In [16]:
# Filter rows where Credit Line is not equal to 'Gift of Charles Lang Freer' and remove them
filtered_df = df[df['Credit Line'] == 'Gift of Charles Lang Freer']


# Save the filtered data to a new Excel file
output_file = 'filtered_collections.xlsx'
filtered_df.to_excel(output_file, index=False)


In [18]:
inconsistent_rows_two = df[(df['Collection'] != 'Freer Gallery of Art Collection')]

print(inconsistent_rows_two['Collection'])

78      Freer Study Collection
79      Freer Study Collection
588     Freer Study Collection
5119    Freer Study Collection
5626    Freer Study Collection
8911    Freer Study Collection
8912    Freer Study Collection
8913    Freer Study Collection
8914    Freer Study Collection
8916    Freer Study Collection
8917    Freer Study Collection
8918    Freer Study Collection
8919    Freer Study Collection
8920    Freer Study Collection
8921    Freer Study Collection
8922    Freer Study Collection
8923    Freer Study Collection
8924    Freer Study Collection
8925    Freer Study Collection
8926    Freer Study Collection
8927    Freer Study Collection
8928    Freer Study Collection
8929    Freer Study Collection
8930    Freer Study Collection
8931    Freer Study Collection
8932    Freer Study Collection
8933    Freer Study Collection
8934    Freer Study Collection
8935    Freer Study Collection
8937    Freer Study Collection
8938    Freer Study Collection
8939    Freer Study Collection
8941    

After filtering, it was observed that some items were purchased by his foundation after Charles's death or collected from other resources. Checking the collection column reveals that some items do not belong to the Freer Gallery of Art but rather to the Freer Study Collection.
The columns "collection" and "credit line" are not meaningful for data analysis, thus, they should be removed.

In [26]:
# Check the unconsistency of "Origin"和"Geography"
df = pd.read_excel('filtered_collections.xlsx')
not_equal_rows = df.loc[df['Origin'] != df['Geography'], ["Name", "Origin", "Geography"]]


# display the results
print(not_equal_rows)


# print(len(not_equal_rows['Name'])) 115

                                                   Name Origin Geography
299           Folio IV from a manuscript of the Gospels    NaN       NaN
319          Folio III from a manuscript of the Gospels    NaN       NaN
412               Cup, small flaring ovoidal; bold foot    NaN       NaN
414                                  Fragment of a bowl    NaN       NaN
748                                Pearls and fragments    NaN       NaN
...                                                 ...    ...       ...
8944  Les conquetes de l'empereur de la chine (in Ch...    NaN       NaN
8945  Les conquetes de l'empereur de la chine (in Ch...    NaN       NaN
8946  Les conquetes de l'empereur de la chine (in Ch...    NaN       NaN
8947  Les conquetes de l'empereur de la chine (in Ch...    NaN       NaN
9455  The Heavenly Ladder: illustration from a Klima...    NaN       NaN

[155 rows x 3 columns]


The artifacts have consistent values between the "geography" and "origin" columns, including those with NaN values.
The statistical characteristics of the "geography" column mirror those of the "origin" column. Considering this, it might be practical to retain only one of them. The column labeled "Restrictions and Rights" was also removed as it did not carry any meaningful statistical significance.

In [28]:
# Read Excel file
df = pd.read_excel('filtered_collections.xlsx')

# List of column names to drop
columns_to_drop = ['Collection', 'Credit Line', 'Origin','Restrictions and Rights','188', '377', '566', '755', '944', '1133']

# Drop specified columns using the drop method
df_cleaned = df.drop(columns=columns_to_drop)

# Save the modified DataFrame back to an Excel file
df_cleaned.to_excel('cleaned_artifacts.xlsx', index=False)

# Display the DataFrame after dropping columns (first few rows)
# print(df_cleaned.head())


In [2]:
import pandas as pd
df = pd.read_excel('cleaned_artifacts.xlsx')
df.shape

(9866, 9)

In data processing, ensuring consistency is paramount for accurate analysis and interpretation. It was observed that the "Material" column contained entries with inconsistent capitalization. After standardizing the case for all entries within this column, the number of unique materials was reduced from 1,479 to 1,474. This adjustment underscores the significance of data normalization, as inconsistencies in capitalization can artificially inflate the count of unique values, potentially leading to erroneous conclusions.

In [3]:
# Material大小写不同

# Select the values of column "Material"
unique_materials = df['Material'].unique()

# Caculate the number
num_unique_materials = len(unique_materials)

print(f"The total number of materials：{num_unique_materials}")

The total number of materials：1479


In [4]:
# 将所有材料名称转换为小写
df['Material'] = df['Material'].str.lower()

# 获取唯一的材料名称
unique_materials = df['Material'].unique()

# 计算唯一材料的数量
num_unique_materials = len(unique_materials)

print(f"The total number of materials after unifying case: {num_unique_materials}")

The total number of materials after unifying case: 1474


In [7]:
#写入表格
df.to_excel('cleaned_artifacts.xlsx', index=False)

In [7]:
import pandas as pd

# 读取 Excel 文件
df = pd.read_excel('merged_artifacts.xlsx')

# 要去除的列名列表
columns_to_drop = ['Collection', 'Credit Line','Origin','188','377','566','755','944','1133']

# 使用 drop 方法去除指定的列
df_cleaned = df.drop(columns=columns_to_drop)

# 选项1: 将修改后的 DataFrame 保存回 Excel 文件
df_cleaned.to_excel('cleaned_artifacts.xlsx', index=False)

# 选项2: 如果你想修改原 DataFrame 而不保存成新文件，可以这样做
# df.drop(columns=columns_to_drop, inplace=True)

# 显示去除列后的 DataFrame（前几行）
print(df_cleaned.head())


                                                Name  \
0                     Portrait of Charles Lang Freer   
1  Cloth-covered box, part of Charles Lang Freer'...   
2                           Miniature mirror (chien)   
3                                      Charm or coin   
4                                               Coin   

                                          Provenance  \
0  To 1903\nRosalind Birnie Philip (1873-1958), L...   
1  To 1918\nPang Yuanji (1864-1949), Shanghai, Ch...   
2  To 1911\nUnidentified owner, Honan-fu, China, ...   
3  From 1907 to 1919\nCharles Lang Freer (1854-19...   
4  From 1907 to 1919\nCharles Lang Freer (1854-19...   

                         Previous custodian or owner                  Type  \
0  Rosalind Birnie Philip (1873-1958) (C.L. Freer...              Painting   
1  Pang Yuanji 龐元濟 (1864-1949) (C.L. Freer source...             Container   
2                     Charles Lang Freer (1854-1919)                Mirror   
3             

In [8]:
df = pd.read_excel('cleaned_artifacts.xlsx')
df.shape
# df.columns

(13722, 10)

In [9]:

df = pd.read_excel('cleaned_artifacts.xlsx')

# 检查每一行是否为空
# `isnull().all(axis=1)` 检查所有列的值是否都是 NaN（空白）
empty_rows = df.isnull().all(axis=1)

# 统计空白行的数量
num_empty_rows = empty_rows.sum()

# 打印空白行的数量
print(f"Number of empty rows: {num_empty_rows}")

# 如果需要，可以显示这些空白行的索引
if num_empty_rows > 0:
    print("Indices of empty rows:", df[empty_rows].index.tolist())
else:
    print("No empty rows found.")


Number of empty rows: 0
No empty rows found.


In [10]:
df = pd.read_excel('correspondence.xlsx')
df.shape

(12567, 3)

In [11]:
empty_rows = df.isnull().all(axis=1)

# 统计空白行的数量
num_empty_rows = empty_rows.sum()

# 打印空白行的数量
print(f"Number of empty rows: {num_empty_rows}")

# 如果需要，可以显示这些空白行的索引
if num_empty_rows > 0:
    print("Indices of empty rows:", df[empty_rows].index.tolist())
else:
    print("No empty rows found.")


Number of empty rows: 0
No empty rows found.
